In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 5.1 CNN 기초

# 5.2 CNN 모델 구현하기

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [3]:
USE_CUDA=torch.cuda.is_available()
DEVICE=torch.device("cuda" if USE_CUDA else "cpu")

In [4]:
EPOCHS=40
BATCH_SIZE=64

In [5]:
train_loader=torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,),(0.3081,))
    ])),
    batch_size=BATCH_SIZE, shuffle=True
)

test_loader=torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                          train=False,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,),(0.3081,))
                          ])),
                          batch_size=BATCH_SIZE,
                          shuffle=True
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw



5x5 의 커널 크기

컨볼루션 계층 2개

In [18]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1=nn.Conv2d(1,10,kernel_size=5)
        self.conv2=nn.Conv2d(10,20, kernel_size=5)
        self.drop=nn.Dropout2d()

        self.fc1=nn.Linear(320, 50)
        self.fc2=nn.Linear(50, 10)

    def forward(self, x):
        x=F.relu(F.max_pool2d(self.conv1(x), 2))
        x=F.relu(F.max_pool2d(self.conv2(x), 2))

        x=x.view(-1, 320)

        x=F.relu(self.fc1(x))
        x=self.drop(x)
        x=self.fc2(x)

        return F.log_softmax(x, dim=1)

In [19]:
model=CNN().to(DEVICE)
optimizer=optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [20]:
def train(model, train_loader, opitmizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target=data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output=model(data)
        loss=F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx%200==0:
            print('Train epoch : {} / {} {} ({:.0f})%\tLoss:{:.6f}'.format(epoch, batch_idx*len(data), len(train_loader.dataset), 100.*batch_idx/len(train_loader), loss.item()))

In [21]:
def evaluate(model, test_loader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for data, target in test_loader:
            data, target=data.to(DEVICE), target.to(DEVICE)
            output=model(data)

            #배치 오차를 합산
            test_loss+=F.cross_entropy(output,target, reduction='sum').item()

            #가장 높은 값을 가진 인덱스가 바로 예측값
            pred=output.max(1, keepdim=True)[1]
            correct+=pred.eq(target.view_as(pred)).sum().item()

    test_loss/=len(test_loader.dataset)
    test_accuracy=100.*correct/len(test_loader.dataset)
    return test_loss, test_accuracy

In [22]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print('[{}] Test Loss: {:.4f}, Accuracy : {:.2f}%'.format(epoch, test_loss, test_accuracy))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Train epoch : 1 / 0 60000 (0)%	Loss:2.308104
Train epoch : 1 / 12800 60000 (21)%	Loss:1.194288
Train epoch : 1 / 25600 60000 (43)%	Loss:0.916958
Train epoch : 1 / 38400 60000 (64)%	Loss:0.916237
Train epoch : 1 / 51200 60000 (85)%	Loss:0.672397
[1] Test Loss: 0.6112, Accuracy : 76.39%
Train epoch : 2 / 0 60000 (0)%	Loss:0.526144
Train epoch : 2 / 12800 60000 (21)%	Loss:0.855989
Train epoch : 2 / 25600 60000 (43)%	Loss:0.729082
Train epoch : 2 / 38400 60000 (64)%	Loss:0.663137
Train epoch : 2 / 51200 60000 (85)%	Loss:0.707805
[2] Test Loss: 0.5304, Accuracy : 79.34%
Train epoch : 3 / 0 60000 (0)%	Loss:0.575296
Train epoch : 3 / 12800 60000 (21)%	Loss:1.011210
Train epoch : 3 / 25600 60000 (43)%	Loss:0.613917
Train epoch : 3 / 38400 60000 (64)%	Loss:0.496573
Train epoch : 3 / 51200 60000 (85)%	Loss:0.380511
[3] Test Loss: 0.5075, Accuracy : 81.16%
Train epoch : 4 / 0 60000 (0)%	Loss:0.657360
Train epoch : 4 / 12800 60000 (21)%	Loss:0.491204
Train epoch : 4 / 25600 60000 (43)%	Loss:0.6118

# Resnet 모델

In [22]:
BATCH_SIZE=128
EPOCHS=300

In [23]:
resnet_train_loader=torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                     train=True,
                     download=True,
                     transform=transforms.Compose([
                         transforms.RandomCrop(32, padding=4),
                         transforms.RandomHorizontalFlip(),
                         transforms.ToTensor(),
                         transforms.Normalize((0.5, 0.5, 0.5),
                                              (0.5, 0.5, 0.5))
                     ])),
                     batch_size=BATCH_SIZE,
                     shuffle=True
)

resnet_test_loader=torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                     train=False,
                     transform=transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize((0.5, 0.5, 0.5),
                                              (0.5, 0.5, 0.5))
                     ])),
                     batch_size=BATCH_SIZE,
                     shuffle=True
)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./.data/cifar-10-python.tar.gz to ./.data


In [24]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1=nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1=nn.BatchNorm2d(planes)
        self.conv2=nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2=nn.BatchNorm2d(planes)

        self.shortcut=nn.Sequential()
        if stride!=1 or in_planes!=planes:
            self.shortcut=nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out=F.relu(self.bn1(self.conv1(x)))
        out=self.bn2(self.conv2(out))
        out+=self.shortcut(x)
        out=F.relu(out)
        return out

In [25]:
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes=16

        self.conv1=nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1=nn.BatchNorm2d(16)
        self.layer1=self._make_layer(16, 2, stride=1)
        self.layer2=self._make_layer(32, 2, stride=2)
        self.layer3=self._make_layer(64, 2, stride=2)
        self.linear=nn.Linear(64, num_classes)

    def _make_layer(self, planes,  num_blocks, stride):
        strides=[stride]+[1]*(num_blocks-1)
        layers=[]
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes=planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out=F.relu(self.bn1(self.conv1(x)))
        out=self.layer1(out)
        out=self.layer2(out)
        out=self.layer3(out)
        out=F.avg_pool2d(out, 8)
        out=out.view(out.size(0), -1)
        out=self.linear(out)
        return out

In [27]:
res_model=ResNet().to(DEVICE)
optimizer=optim.SGD(res_model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0005)
scheduler=optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

In [28]:
for epoch in range(1, EPOCHS+1):
    scheduler.step()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print('[{}] Test Loss : {:.4f}, Accuracy : {:.2f}%'.format(epoch, test_loss, test_accuracy))

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  w

Train epoch : 1 / 0 60000 (0)%	Loss:0.235540
Train epoch : 1 / 12800 60000 (21)%	Loss:0.258228
Train epoch : 1 / 25600 60000 (43)%	Loss:0.261930
Train epoch : 1 / 38400 60000 (64)%	Loss:0.232354
Train epoch : 1 / 51200 60000 (85)%	Loss:0.193414
[1] Test Loss : 0.2881, Accuracy : 89.74%
Train epoch : 2 / 0 60000 (0)%	Loss:0.427865
Train epoch : 2 / 12800 60000 (21)%	Loss:0.335132
Train epoch : 2 / 25600 60000 (43)%	Loss:0.432580
Train epoch : 2 / 38400 60000 (64)%	Loss:0.188695
Train epoch : 2 / 51200 60000 (85)%	Loss:0.278517
[2] Test Loss : 0.2881, Accuracy : 89.74%
Train epoch : 3 / 0 60000 (0)%	Loss:0.310427
Train epoch : 3 / 12800 60000 (21)%	Loss:0.263496
Train epoch : 3 / 25600 60000 (43)%	Loss:0.218157
Train epoch : 3 / 38400 60000 (64)%	Loss:0.273454
Train epoch : 3 / 51200 60000 (85)%	Loss:0.184666
[3] Test Loss : 0.2881, Accuracy : 89.74%
Train epoch : 4 / 0 60000 (0)%	Loss:0.309610
Train epoch : 4 / 12800 60000 (21)%	Loss:0.419825
Train epoch : 4 / 25600 60000 (43)%	Loss:0.2